In [1]:
import torch
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image
import torch.nn.functional as F

bs = 100
# MNIST Dataset
train_dataset = datasets.MNIST(root='./mnist_data/', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='./mnist_data/', train=False, transform=transforms.ToTensor(), download=False)

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=bs, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=bs, shuffle=False)

In [2]:
from VAE import VAE
vae = VAE(size=784)
if torch.cuda.is_available():
    vae.cuda()

In [3]:
optimizer = optim.Adam(vae.parameters())
# return reconstruction error + KL divergence losses
def loss_function(recon_x, x, mu, log_var):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
    KLD = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    return BCE + KLD

In [4]:
def train(epoch):
    vae.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.cuda()
        optimizer.zero_grad()
        
        recon_batch, mu, log_var = vae(data)
        loss = loss_function(recon_batch, data, mu, log_var)
        
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item() / len(data)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))

In [5]:
def test():
    vae.eval()
    test_loss= 0
    with torch.no_grad():
        for data, _ in test_loader:
            data = data.cuda()
            recon, mu, log_var = vae(data)
            
            # sum up batch loss
            test_loss += loss_function(recon, data, mu, log_var).item()
        
    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [6]:
for epoch in range(1, 3):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 543.752422
Train Epoch: 1 [10000/60000 (17%)]	Loss: 200.941777
Train Epoch: 1 [20000/60000 (33%)]	Loss: 183.495176
Train Epoch: 1 [30000/60000 (50%)]	Loss: 168.650430
Train Epoch: 1 [40000/60000 (67%)]	Loss: 155.495762
Train Epoch: 1 [50000/60000 (83%)]	Loss: 149.102402
====> Epoch: 1 Average loss: 173.2168
====> Test set loss: 141.5618
Train Epoch: 2 [0/60000 (0%)]	Loss: 139.749736
Train Epoch: 2 [10000/60000 (17%)]	Loss: 143.488506
Train Epoch: 2 [20000/60000 (33%)]	Loss: 141.246895
Train Epoch: 2 [30000/60000 (50%)]	Loss: 137.067588
Train Epoch: 2 [40000/60000 (67%)]	Loss: 132.077500
Train Epoch: 2 [50000/60000 (83%)]	Loss: 126.497998
====> Epoch: 2 Average loss: 135.5013
====> Test set loss: 129.9040
